# Тесты эстиматоров взаимной информации

## Преамбула

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
font = {'family' : 'DejaVu Sans',
        'size'   : 18}

matplotlib.rc('font', **font)

### Импортирование модуля

In [ ]:
import sys
sys.path.insert(0, './py')

In [ ]:
import mutinfo.estimators.mutual_information as mi_estiamtors
from mutinfo.utils.dependent_norm import multivariate_normal_from_MI

In [ ]:
global_n_jobs = 32

## Нормальный случайный вектор

In [ ]:
# Исследуемые значения взаимной информации.
MI = [0.0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0, 3.0, 5.0, 7.0, 10.0]
n_exps = len(MI)

# Число экземпляров и размерности векторов X и Y.
n_samples = 1000
X_dimension = 2
Y_dimension = 2

In [ ]:
# Оценки взаимной информации.
estimated_MI = []

# Проведение тестов.
for n_exp in range(n_exps):
    # Генерация.
    random_variable = multivariate_normal_from_MI(X_dimension, Y_dimension, MI[n_exp])
    X_Y = random_variable.rvs(n_samples)
    X = X_Y[:, 0:X_dimension]
    Y = X_Y[:, X_dimension:X_dimension + Y_dimension]
    
    # Оценка энтропии.
    mi_estimator = mi_estiamtors.MutualInfoEstimator(Y_is_discrette = False, n_jobs = global_n_jobs)
    mi_estimator.fit(X, Y, verbose = 10)
    estimated_MI.append(mi_estimator.predict(X, Y, verbose = 10))

In [ ]:
estimated_MI_mean = np.array([estimated_MI[index][0] for index in range(n_exps)])
estimated_MI_std  = np.array([estimated_MI[index][1] for index in range(n_exps)])

In [ ]:
fig_normal, ax_normal = plt.subplots()

fig_normal.set_figheight(11)
fig_normal.set_figwidth(16)

# Сетка.
ax_normal.grid(color='#000000', alpha=0.15, linestyle='-', linewidth=1, which='major')
ax_normal.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')

ax_normal.set_title("Тест на нормальном векторе")
ax_normal.set_xlabel("$I(X,Y)$")
ax_normal.set_ylabel("$\\hat I(X,Y)$")

#ax_normal.set_yscale('log')
#ax_normal.set_xscale('log')

ax_normal.plot(MI, MI, label="$I(X,Y)$")
ax_normal.plot(MI, estimated_MI_mean, label="$\\hat I(X,Y)$")
ax_normal.fill_between(MI, estimated_MI_mean + estimated_MI_std, estimated_MI_mean - estimated_MI_std, alpha=0.2)

ax_normal.legend(loc='upper right')
plt.show();